<a href="https://colab.research.google.com/github/MengOonLee/Deep_learning/blob/master/TensorFlow/Customise/SubclassCustom/ResidualMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programming Assignment

## Residual network

### Instructions

In this notebook, you will use the model subclassing API together with custom layers to create a residual network architecture. You will then train your custom model on the Fashion-MNIST dataset by using a custom training loop and implementing the automatic differentiation tools in Tensorflow to calculate the gradients for backpropagation.

Some code cells are provided you in the notebook. You should avoid editing provided code, and make sure to execute the cells in order to avoid unexpected errors. Some cells begin with the line:

`#### GRADED CELL ####`

Don't move or edit this first line - this is what the automatic grader looks for to recognise graded cells. These cells require you to write your own code to complete them, and are automatically graded when you submit the notebook. Don't edit the function name or signature provided in these cells, otherwise the automatic grader might not function properly. Inside these graded cells, you can use any functions or classes that are imported below, but make sure you don't use any variables that are outside the scope of the function.

### How to submit

Complete all the tasks you are asked for in the worksheet. When you have finished and are happy with your code, press the **Submit Assignment** button at the top of this notebook.

### Let's get started!

We'll start running some imports, and loading the dataset. Do not edit the existing imports in the following cell. If you would like to make further Tensorflow imports, you should add them here.

![Fashion-MNIST overview image](https://github.com/MengOonLee/Deep_learning/blob/master/TensorFlow/image/fashion_mnist/fashion_mnist.png?raw=1)

#### The Fashion-MNIST dataset

In this assignment, you will use the [Fashion-MNIST dataset](https://github.com/zalandoresearch/fashion-mnist). It consists of a training set of 60,000 images of fashion items with corresponding labels, and a test set of 10,000 images. The images have been normalised and centred. The dataset is frequently used in machine learning research, especially as a drop-in replacement for the MNIST dataset.

- H. Xiao, K. Rasul, and R. Vollgraf. "Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning Algorithms." arXiv:1708.07747, August 2017.

Your goal is to construct a ResNet model that classifies images of fashion items into one of 10 classes.

In [1]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
tf.keras.utils.set_random_seed(seed=42)
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

#### Load the dataset

For this programming assignment, we will take a smaller sample of the dataset to reduce the training time.

In [2]:
# Load and preprocess the Fashion-MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets\
    .fashion_mnist.load_data()

inputs = tf.keras.Input(shape=(28, 28), dtype=tf.float32)
h = tf.keras.layers.Rescaling(scale=1./255)(inputs)
outputs = tf.keras.layers.Reshape(target_shape=(28, 28, 1))(h)
preprocessing_model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Create Dataset objects for the training and test sets
def get_ds(x, y, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices(tensors=(x, y))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(y))
    ds = ds.batch(batch_size=64)
    ds = ds.map(map_func=lambda x, y: (preprocessing_model(x), y),
        num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

ds_train = get_ds(x=x_train, y=y_train, shuffle=True)
ds_test = get_ds(x=x_test, y=y_test)
ds_test.element_spec

(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.uint8, name=None))

#### Create custom layers for the residual blocks

You should now create a first custom layer for a residual block of your network. Using layer subclassing, build your custom layer according to the following spec:

* The custom layer class should have `__init__`, `build` and `call` methods. The `__init__` method has been completed for you. It calls the base `Layer` class initializer, passing on any keyword arguments
* The `build` method should create the layers. It will take an `input_shape` argument, and should extract the number of filters from this argument. It should create:
    * A BatchNormalization layer: this will be the first layer in the block, so should use its `input_shape` keyword argument
    * A Conv2D layer with the same number of filters as the layer input, a 3x3 kernel size, `'SAME'` padding, and no activation function
    * Another BatchNormalization layer
    * Another Conv2D layer, again with the same number of filters as the layer input, a 3x3 kernel size, `'SAME'` padding, and no activation function
* The `call` method should then process the input through the layers:
    * The first BatchNormalization layer: ensure to set the `training` keyword argument
    * A `tf.nn.relu` activation function
    * The first Conv2D layer
    * The second BatchNormalization layer: ensure to set the `training` keyword argument
    * Another `tf.nn.relu` activation function
    * The second Conv2D layer
    * It should then add the layer inputs to the output of the second Conv2D layer. This is the final layer output

In [3]:
#### GRADED CELL ####
# Complete the following class.
# Make sure to not change the class or method names or arguments.

class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.batchnorm_2 = tf.keras.layers.BatchNormalization()

    def build(self, input_shape):
        """
        This method should build the layers according to the above
        specification. Make sure to use the input_shape argument to get the
        correct number of filters, and to set the input_shape of the first
        layer in the block.
        """
        self.batchnorm_1 = tf.keras.layers.BatchNormalization(
            input_shape=input_shape)
        self.conv2d_1 = tf.keras.layers.Conv2D(filters=input_shape[-1],
            kernel_size=(3, 3), padding='same', activation=None,
            kernel_regularizer=tf.keras.regularizers.L2(l2=1e-5))
        self.conv2d_2 = tf.keras.layers.Conv2D(filters=input_shape[-1],
            kernel_size=(3, 3), padding='same', activation=None,
            kernel_regularizer=tf.keras.regularizers.L2(l2=1e-5))

    def call(self, inputs, training=False):
        """
        This method should contain the code for calling the layer according to
        the above specification, using the layer objects set up
        in the build method.
        """
        h = self.batchnorm_1(inputs=inputs, training=training)
        h = tf.nn.relu(features=h)
        h = self.conv2d_1(inputs=h)
        h = self.batchnorm_2(inputs=h, training=training)
        h = tf.nn.relu(features=h)
        h = self.conv2d_2(inputs=h)
        return tf.keras.layers.Add()([inputs, h])

# Test your custom layer - the following should create a model using your layer
inputs = tf.keras.Input(shape=(28, 28, 1))
outputs = ResidualBlock(name='residual_block')(inputs)
test_model = tf.keras.Model(inputs=inputs, outputs=outputs)
test_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 residual_block (ResidualBl  (None, 28, 28, 1)         28        
 ock)                                                            
                                                                 
Total params: 28 (112.00 Byte)
Trainable params: 24 (96.00 Byte)
Non-trainable params: 4 (16.00 Byte)
_________________________________________________________________


You should now create a second custom layer for a residual block of your network. This layer will be used to change the number of filters within the block. Using layer subclassing, build your custom layer according to the following spec:

* The custom layer class should have `__init__`, `build` and `call` methods
* The class initialiser should call the base `Layer` class initializer, passing on any keyword arguments. It should also accept a `out_filters` argument, and save it as a class attribute
* The `build` method should create the layers. It will take an `input_shape` argument, and should extract the number of input filters from this argument. It should create:
    * A BatchNormalization layer: this will be the first layer in the block, so should use its `input shape` keyword argument
    * A Conv2D layer with the same number of filters as the layer input, a 3x3 kernel size, `"SAME"` padding, and no activation function
    * Another BatchNormalization layer
    * Another Conv2D layer with `out_filters` number of filters, a 3x3 kernel size, `"SAME"` padding, and no activation function
    * A final Conv2D layer with `out_filters` number of filters, a 1x1 kernel size, and no activation function
* The `call` method should then process the input through the layers:
    * The first BatchNormalization layer: ensure to set the `training` keyword argument
    * A `tf.nn.relu` activation function
    * The first Conv2D layer
    * The second BatchNormalization layer: ensure to set the `training` keyword argument
    * Another `tf.nn.relu` activation function
    * The second Conv2D layer
    * It should then take the layer inputs, pass it through the final 1x1 Conv2D layer, and add to the output of the second Conv2D layer. This is the final layer output

In [4]:
#### GRADED CELL ####
# Complete the following class.
# Make sure to not change the class or method names or arguments.

class FiltersChangeResidualBlock(tf.keras.layers.Layer):
    def __init__(self, out_filters, *args, **kwargs):
        """
        The class initialiser should call the base class initialiser, passing
        any keyword arguments along. It should also set the number of filters
        as a class attribute.
        """
        super().__init__(*args, **kwargs)
        self.out_filters = out_filters
        self.batchnorm_2 = tf.keras.layers.BatchNormalization()
        self.conv2d_2 = tf.keras.layers.Conv2D(filters=out_filters,
            kernel_size=(3, 3), padding='same', activation=None,
            kernel_regularizer=tf.keras.regularizers.L2(l2=1e-5))
        self.conv2d_3 = tf.keras.layers.Conv2D(filters=out_filters,
            kernel_size=(1, 1), activation=None,
            kernel_regularizer=tf.keras.regularizers.L2(l2=1e-5))

    def build(self, input_shape):
        """
        This method should build the layers according to the above
        specification. Make sure to use the input_shape argument to get the
        correct number of filters, and to set the input_shape of the first
        layer in the block.
        """
        self.batchnorm_1 = tf.keras.layers.BatchNormalization(
            input_shape=input_shape)
        self.conv2d_1 = tf.keras.layers.Conv2D(filters=input_shape[-1],
            kernel_size=(3, 3), padding='same', activation=None,
            kernel_regularizer=tf.keras.regularizers.L2(l2=1e-5))

    def call(self, inputs, training=False):
        """
        This method should contain the code for calling the layer according to
        the above specification, using the layer objects set up in the build
        method.
        """
        h1 = self.batchnorm_1(inputs=inputs, training=training)
        h1 = tf.nn.relu(features=h1)
        h1 = self.conv2d_1(inputs=h1)
        h1 = self.batchnorm_2(inputs=h1, training=training)
        h1 = tf.nn.relu(features=h1)
        h1 = self.conv2d_2(inputs=h1)
        h2 = self.conv2d_3(inputs=inputs)
        return tf.keras.layers.Add()([h1, h2])

# Test your custom layer - the following should create a model using your layer
inputs = tf.keras.Input(shape=(32, 32, 3))
outputs = FiltersChangeResidualBlock(name='fc_resnet_block',
    out_filters=16)(inputs)
test_model = tf.keras.Model(inputs=inputs, outputs=outputs)
test_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 fc_resnet_block (FiltersCh  (None, 32, 32, 16)        620       
 angeResidualBlock)                                              
                                                                 
Total params: 620 (2.42 KB)
Trainable params: 608 (2.38 KB)
Non-trainable params: 12 (48.00 Byte)
_________________________________________________________________


#### Create a custom model that integrates the residual blocks

You are now ready to build your ResNet model. Using model subclassing, build your model according to the following spec:

* The custom model class should have `__init__` and `call` methods.
* The class initialiser should call the base `Model` class initializer, passing on any keyword arguments. It should create the model layers:
    * The first Conv2D layer, with 32 filters, a 7x7 kernel and stride of 2.
    * A `ResidualBlock` layer.
    * The second Conv2D layer, with 32 filters, a 3x3 kernel and stride of 2.
    * A `FiltersChangeResidualBlock` layer, with 64 output filters.
    * A Flatten layer
    * A final Dense layer, with a 10-way softmax output
* The `call` method should then process the input through the layers in the order given above. Ensure to pass the `training` keyword argument to the residual blocks, to ensure the correct mode of operation for the batch norm layers.

In total, your neural network should have six layers (counting each residual block as one layer).

In [5]:
#### GRADED CELL ####
# Complete the following class.
# Make sure to not change the class or method names or arguments.

@tf.keras.saving.register_keras_serializable()
class CustomModel(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        """
        The class initialiser should call the base class initialiser, passing
        any keyword arguments along. It should also create the layers of the
        network according to the above specification.
        """
        super().__init__(*args, **kwargs)

        self.conv2d_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(7, 7),
            strides=2, kernel_regularizer=tf.keras.regularizers.L2(l2=1e-5))
        self.residual_block = ResidualBlock()
        self.conv2d_2 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3),
            strides=2, kernel_regularizer=tf.keras.regularizers.L2(l2=1e-5))
        self.filters_change_residual_block = FiltersChangeResidualBlock(
            out_filters=64)
        self.flatten = tf.keras.layers.Flatten()
        self.dropout = tf.keras.layers.Dropout(rate=0.2)
        self.dense = tf.keras.layers.Dense(units=10,
            activation=tf.keras.activations.softmax)

    def call(self, inputs, training=False):
        """
        This method should contain the code for calling the layer according to
        the above specification, using the layer objects set up in the
        initialiser.
        """
        h = self.conv2d_1(inputs=inputs)
        h = self.residual_block(inputs=h,
            training=training)
        h = self.conv2d_2(inputs=h)
        h = self.filters_change_residual_block(inputs=h,
            training=training)
        h = self.flatten(inputs=h)
        h = self.dropout(inputs=h, training=training)
        return self.dense(inputs=h)

#### Define the optimizer, loss function and grad function

We will use the Adam optimizer with a learning rate of 0.001, and the sparse categorical cross entropy function.

You should now create the `train` function that will compute the forward and backward pass, and return the loss value and gradients that will be used in your custom training loop:

* The `train` function takes a model instance, inputs, targets and the loss object above as arguments
* The function should use a `tf.GradientTape` context to compute the forward pass and calculate the loss
* The function should compute the gradient of the loss with respect to the model's trainable variables
* The function should return a tuple of two elements: the loss value, and a list of gradients

In [6]:
@tf.function
def train_step(self, data):
    """
    This function should compute the loss and gradients of your model,
    corresponding to the inputs x and targets y provided. It should return
    the loss and accuracy.
    """
    x, y = data

    with tf.GradientTape() as tape:
        y_pred = self(inputs=x, training=True)
        loss = self.compute_loss(y=y, y_pred=y_pred)

    trainable_vars = self.trainable_variables
    grads = tape.gradient(target=loss, sources=trainable_vars)
    self.optimizer.apply_gradients(grads_and_vars=zip(
        grads, trainable_vars))

    for m in self.metrics:
        if m.name=='loss':
            m.update_state(values=loss)
        else:
            m.update_state(y_true=y, y_pred=y_pred)

    return {m.name: m.result() for m in self.metrics}

@tf.function
def test_step(self, data):
    x, y = data

    y_pred = self(inputs=x, training=False)
    loss = self.compute_loss(y=y, y_pred=y_pred)

    for m in self.metrics:
        if m.name=='loss':
            m.update_state(values=loss)
        else:
            m.update_state(y_true=y, y_pred=y_pred)

    return {m.name: m.result() for m in self.metrics}

CustomModel.metrics=[
    tf.keras.metrics.Mean(name='loss'),
    tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')
]
CustomModel.train_step = train_step
CustomModel.test_step= test_step

#### Define the custom training loop

You should now write a custom training loop. Complete the following function, according to the spec:

* The function takes the following arguments:
    * `model`: an instance of your custom model
    * `num_epochs`: integer number of epochs to train the model
    * `train_dataset`: a `tf.data.Dataset` object for the training data
    * `valid_dataset`: a `tf.data.Dataset` object for the testing data
   
* Your function should train the model for the given number of epochs, using the `train` above, that returns the loss and gradients for given model, inputs and targets for each training batch, and updating the model parameters using the optimizer object as created `optimizer.apply_gradients`.
* Your function should collect the mean `loss`, a sparse categorical cross entropy object as created above and accuracy values over the epoch, and return a dataframe of the two metrics; the first for the loss values per epoch, the second for the accuracy values per epoch.

You may also want to print out the loss and accuracy at each epoch during the training.

In [ ]:
#### GRADED CELL ####
# Complete the following function.
# Make sure to not change the function name or arguments.

start_time = time.time()
# Create the model, optimizer
resnet_model = CustomModel()
resnet_model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(name='loss'),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
    patience=1, factor=0.9)

history = resnet_model.fit(x=ds_train, validation_data=ds_test,
    epochs=30, callbacks=[reduce_lr], verbose=0)
print('Training duration: %.2fs'%(time.time() - start_time))

# Train the model for 10 epochs
df_history = pd.DataFrame(history.history)
df_history['epoch'] = df_history.index + 1

# Plot the learning curves
fig, axes = plt.subplots(nrows=2, sharex=True, figsize=(12, 8))
fig.suptitle(t='Training Metrics')
axes[1].set_xlabel(xlabel='epoch')
for ax, metric in zip(axes, ['loss', 'accuracy']):
    g = sns.lineplot(ax=ax, label='train', data=df_history,
        x='epoch', y=f"{metric}")
    g = sns.lineplot(ax=ax, label="val", data=df_history,
        x='epoch', y=f'val_{metric}')
    ax.legend(loc='best')
    ax.set_ylabel(ylabel=f'{metric}')
    ax.set_title(label=f'{metric} vs. epoch')
fig.tight_layout()
plt.show()

loss, acc = model.evaluate(x=ds_test, verbose=0)
print(f'Test loss: {loss:.2f}, Test accuracy: {acc:.2f}')

#### Model predictions

Let's see some model predictions! We will randomly select four images from the test data, and display the image and label for each.

For each test image, model's prediction (the label with maximum probability) is shown, together with a plot showing the model's categorical distribution.

In [ ]:
inputs = preprocessing_model.input
outputs = resnet_model(preprocessing_model(inputs))
inference_model = tf.keras.Model(inputs=inputs, outputs=outputs)

os.makedirs('models', exist_ok=True)
inference_model.save('models/resnet_model.keras')
inference_model = tf.keras.models.load_model('models/resnet_model.keras')

# Run this cell to get model predictions on randomly selected test images
rand_inx = np.random.choice(a=x_test.shape[0], size=4)
x_rand = x_test[rand_inx, ...]
y_rand = y_test[rand_inx, ...]
preds = inference_model(inputs=x_rand)

# Get dataset labels
labels = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(16, 12))
fig.subplots_adjust(hspace=0.5, wspace=-0.2)
for i, (y_pred, x, y) in enumerate(
        zip(preds, x_rand, y_rand)):
    axes[i, 0].imshow(np.squeeze(a=x))
    axes[i, 0].get_xaxis().set_visible(b=False)
    axes[i, 0].get_yaxis().set_visible(b=False)
    axes[i, 0].text(x=5., y=-2., s=f"Class {y} ({labels[y]})")
    axes[i, 1].bar(x=np.arange(stop=len(y_pred)), height=y_pred)
    axes[i, 1].set_xticks(ticks=np.arange(stop=len(y_pred)))
    axes[i, 1].set_xticklabels(labels=labels, rotation=0)
    pred_inx = np.argmax(a=y_pred)
    axes[i, 1].set_title(label=f"Categorical distribution.\
Model prediction: {labels[pred_inx]}")
plt.show()

Congratulations for completing this programming assignment! You're now ready to move on to the capstone project for this course.